<a href="https://colab.research.google.com/github/marcomag416/MLDL/blob/main/evaluate_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from PIL import Image
import sys
import requests
from zipfile import ZipFile
from io import BytesIO

In [2]:
#download and extract project repository

url= "https://github.com/marcomag416/MLDL/archive/refs/heads/main.zip"

# Send a GET request to the URL
response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    #print(response.content)
    # Open the downloaded bytes and extract them
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('./')
    print('Download and extraction complete!')

sys.path.insert(0, './MLDL-main')

Download and extraction complete!


In [3]:
#download cityscapes dataset
from google.colab import drive
drive.mount('/content/drive')

cityscape_dataset_path = "./dataset/Cityscapes/Cityspaces"

#extract zip file
if not os.path.exists(cityscape_dataset_path):
  print("Extracting dataset...")
  with ZipFile("/content/drive/MyDrive/Colab Notebooks/dataset/Cityscapes.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset")



Mounted at /content/drive
Extracting dataset...


# Evaluation


In [4]:
#import from packages
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn

#other imports
from models.bisenet.build_bisenet import BiSeNet

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"


## Dataset

In [5]:
class Cityscapes(Dataset):
    def __init__(self, root_dir, split, transforms=None, label_type='gtFine_labelTrainIds'):
        self.root_dir = root_dir
        self.split = split
        self.transforms = transforms
        self.label_type = label_type

        self.images_dir = f"{root_dir}/images/{split}"
        self.labels_dir = f"{root_dir}/gtFine/{split}"

        self.image_paths = []
        self.label_paths = []

        # Manually iterate over directories
        cities = [city for city in os.listdir(self.images_dir) if os.path.isdir(f"{self.images_dir}/{city}")]
        for city in cities:
            img_dir_city = f"{self.images_dir}/{city}"
            lbl_dir_city = f"{self.labels_dir}/{city}"

            if not os.path.isdir(img_dir_city) or not os.path.isdir(lbl_dir_city):
                continue

            for img_file in os.listdir(img_dir_city):
                if img_file.endswith('_leftImg8bit.png'):
                    img_path = f"{img_dir_city}/{img_file}"
                    lbl_file = img_file.replace('_leftImg8bit.png', f'_{self.label_type}.png')
                    lbl_path = f"{lbl_dir_city}/{lbl_file}"

                    if os.path.isfile(img_path) and os.path.isfile(lbl_path):
                        self.image_paths.append(img_path)
                        self.label_paths.append(lbl_path)
                    else:
                        print(f"Warning: Image or label file not found for {img_file}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        lbl_path = self.label_paths[idx]

        image = Image.open(img_path).convert('RGB')
        label = Image.open(lbl_path)

        image = np.array(image)
        label = np.array(label)

        if self.transforms:
            augmented = self.transforms(image=image, mask=label)
            image, label = augmented['image'], augmented['mask']

        return image, label


# Example usage
image_transforms = A.Compose([
    A.Resize(512, 1024),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

city_dataset_val = Cityscapes(root_dir=cityscape_dataset_path, split='val', transforms=image_transforms)

print(f"Cityscapes_Dataset_val size: {len(city_dataset_val)}")

Cityscapes_Dataset_val size: 500


## Test function

In [6]:
def compute_iou(pred, target, num_classes):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = (pred == cls)
        target_inds = (target == cls)
        intersection = (pred_inds[target_inds]).sum().item()
        union = pred_inds.sum().item() + target_inds.sum().item() - intersection
        if union == 0:
            ious.append(float('nan'))  # If there is no union, set IoU to NaN
        else:
            ious.append(intersection / union)

    return np.array(ious)

def eval(model, dataloader, loss_fn, device, num_classes=19):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    total = 0
    all_ious = []  # List to store IoUs for each batch

    with torch.no_grad():  # Disable gradient calculation during inference
        for inputs_test, targets_test in dataloader:
            inputs_test, targets_test = inputs_test.to(device), targets_test.to(device, dtype=torch.long)

            outputs_test = model(inputs_test)  # Forward pass
            loss = loss_fn(outputs_test, targets_test)  # Calculate the loss

            test_loss += loss.item() * inputs_test.size(0)  # Accumulate the total loss
            _, predicted_test = outputs_test.max(1)
            total += targets_test.size(0)

            # Compute IoU for this batch
            batch_ious = compute_iou(predicted_test, targets_test, num_classes)
            all_ious.append(batch_ious)

    # Calculate average loss
    avg_loss = test_loss / total

    # Calculate mean IoU
    all_ious = np.array(all_ious)
    mean_iou = np.nanmean(all_ious, axis=0)  # Mean IoU for each class
    miou = np.nanmean(mean_iou)  # Mean IoU across all classes

    return avg_loss, miou, mean_iou

## create model and load weights

In [7]:
pretrained_weights_paths = [
    "./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/gaussian_jitter_epoch19.pth",
    "./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/gaussian_jitter_epoch24.pth",
    "./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/no_transform_epoch19.pth",
    "./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/guassian_horizFlip_epoch9.pth",
    "./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/guassian_horizFlip_epoch34.pth",
    "./drive/MyDrive/Colab Notebooks/model_weights/domain_adaptation_right/pretrained19_shallow_paper_step2099.pth",
    "./drive/MyDrive/Colab Notebooks/model_weights/domain_adaptation_right/pretrained19_paper_step3999.pth",
]

In [8]:
# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

context_path = 'resnet18'

city_dataloader_val = DataLoader(city_dataset_val, batch_size=8, shuffle=False, num_workers=4)

# Initialize the model
model = BiSeNet(num_classes=19, context_path=context_path).to(device)
seg_loss_fn = nn.CrossEntropyLoss(ignore_index=255)



Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 92.6MB/s]


In [9]:
for pretrained_weights_path in pretrained_weights_paths:
  model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))

  avg_loss, miou, IoUs = eval(model, city_dataloader_val, seg_loss_fn, device=device)
  print(f"Weight path: {pretrained_weights_path}, Loss: {avg_loss}, mIoU: {miou*100:.2f}%")
  print(f"\t IoU per class: {IoUs}\n")

Weight path: ./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/gaussian_jitter_epoch19.pth, Loss: 0.9514270610809327, mIoU: 22.76%
	 IoU per class: [0.64938667 0.05528171 0.63537795 0.093507   0.08048576 0.17290462
 0.12822551 0.03756267 0.75062853 0.17082393 0.62084863 0.21945976
 0.00282181 0.62840024 0.06023732 0.         0.00092337 0.01177182
 0.00524737]

Weight path: ./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/gaussian_jitter_epoch24.pth, Loss: 1.0272702436447143, mIoU: 20.50%
	 IoU per class: [6.17880781e-01 1.33314043e-01 5.52801540e-01 9.33224893e-02
 9.18102062e-02 9.27035285e-02 6.42167175e-02 3.86552210e-02
 7.47825724e-01 1.45570470e-01 6.52817615e-01 1.17293114e-01
 4.07029042e-04 4.88401180e-01 4.12332670e-02 1.38863429e-02
 0.00000000e+00 2.20502069e-03 0.00000000e+00]

Weight path: ./drive/MyDrive/Colab Notebooks/model_weights/bisenet_gta/no_transform_epoch19.pth, Loss: 1.623073085784912, mIoU: 16.74%
	 IoU per class: [0.3994889  0.01977984 0.5013